In [4]:
from pymongo import MongoClient

In [6]:
client = MongoClient("localhost:27017")

In [7]:
client.list_database_names()

['Quotes',
 'Species',
 'admin',
 'companies',
 'config',
 'final',
 'geo',
 'labflask',
 'local']

In [8]:
companies = client.get_database("companies")

In [11]:
companies.list_collection_names()

['companies', 'offices']

In [13]:
comp = companies.get_collection("companies")

In [23]:
comp.find_one({"total_money_raised":{"$gt":"$1M"}})

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di